In [1]:
import sys
print(sys.version)

3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]


Dependencies

In [2]:
# Systems
import time
import sys
import os

# Sensor input
from  GPS_VK2828U7G5LF import init_gps_GPRMC,read_gps
from mpu6050 import mpu6050
import ADS1115

# computer vision
import cv2
import numpy as np
from PIL import Image
#import PIL

# Hardware
import board
import busio
import Jetson.GPIO as GPIO

# mult thread processing
import threading

#Display clontrol library

import ipywidgets
#from ipywidgets import Button, HBox, VBox, Layout, Label, IntSlider
import ipywidgets as widgets
import IPython #.display import display, clear_output


#PWM driver library
from pca9685_driver import Device

# servo config
servo = Device(0x40,1)
servo.set_pwm_frequency(50)

time.sleep(0.5)

#control steering position fouction
# Lateral control
servo.set_pwm(0,307)
#range 245x370 (307 to center) (245 all to the right) (370 all to the left)

#control motor power
# Longitudinal control
servo.set_pwm(1,330)
#to stop 330
#to go forward 345
#to go backward 320 to times

# GPS init
init_gps_GPRMC()

# IMU init
#imu = mpu6050(0x68)

# ADC init
#ads = ADS1115.ADS1115()


Control

In [3]:
# camera control
#cap = cv2.VideoCapture('nvarguscamerasrc ! video/x-raw(memory:NVMM), width=3280, height=2464, format=NV12,framerate=21/1 ! nvvidconv flip-method=0 ! video/x-raw, width=320, height=240, format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink')
    
# lateral control with buttons

# left
button1 = widgets.Button(description='left',
           layout=widgets.Layout(width='33%', height='80px'))
# s_forward
button2 = widgets.Button(description='s_forward',
           layout=widgets.Layout(width='33%', height='80px'))
# right
button3 = widgets.Button(description='right',
           layout=widgets.Layout(width='33%', height='80px'))

hbox1 = widgets.HBox([button1,button2,button3])

# lateral control with slider

slider1 = widgets.FloatSlider(
            value=307.0,
            min=245.0,#370.0
            max=370.0,#245.0
            step=1.0,
            description='steering:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )

vbox_1_Lateral_control_display = widgets.VBox([widgets.Label('camera:'),widgets.Label('Lateral Control:'), slider1, hbox1])# create the Lateral_control_display


# longitudinal control

# reverse
button4 = widgets.Button(description='reverse',
           layout=widgets.Layout(width='33%', height='80px'))
# stop
button5 = widgets.Button(description='stop',
            button_style= 'danger',
            layout=widgets.Layout(width='33%', height='80px'))
# p_forward
button6 = widgets.Button(description='p_forward',
           layout=widgets.Layout(width='33%', height='80px'))

hbox2 = widgets.HBox([button4,button5,button6])

# longitudinal control with slider max=350
slider2 = widgets.FloatSlider(
            value=330.0,
            min=320.0,
            max=350.0,
            step=1.0,
            description='power:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )

vbox_2_Longitudinal_control_display = widgets.VBox([widgets.Label('camera:'),widgets.Label('Longitudinal Control:'), slider2, hbox2])# create the Lateral_control_display

output_1 = widgets.Output()
output_2 = widgets.Output()
output_3 = widgets.Output()

hbox3 = widgets.HBox([output_2,],layout=widgets.Layout(width='320px', height='250px'))
hbox5 = widgets.HBox([output_3,],layout=widgets.Layout(width='600px', height='400px'))

#clear output button
button7 = widgets.Button(description='clear',
           layout=widgets.Layout(width='70px', height='70px'))

# manual/auto
button8 = widgets.ToggleButton(description='Auto',
            button_style='info', 
            layout=widgets.Layout(width='70px', height='70px'))

vbox_3 = widgets.VBox([button7,button8])

hbox4 = widgets.HBox([vbox_1_Lateral_control_display,vbox_2_Longitudinal_control_display,vbox_3]) 

# End display all
IPython.display.display(hbox4,hbox3,hbox5,output_1)# End display all

# Now all the functions to control

# steering functions

def on_button1_clicked(b):
    with output_1:
        value = 370
        print("left:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button1.on_click(on_button1_clicked)

def on_button2_clicked(b):
    with output_1:
        value = 307
        print("s_forward:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button2.on_click(on_button2_clicked)

def on_button3_clicked(b):
    with output_1:
        value = 245
        print("right:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button3.on_click(on_button3_clicked)

def on_slider1_change(change):
    with output_1:
        print("steering:",change['new'])
        servo.set_pwm(0,((int)(change['new'])))

slider1.observe(on_slider1_change, names='value')


#control motor power functions

# reverse button control
def on_button4_clicked(b):
    with output_1:
        value = 320
        print("reverse:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button4.on_click(on_button4_clicked)

#stop button function
def on_button5_clicked(b):
    with output_1:
        value = 330
        print("stop:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button5.on_click(on_button5_clicked)

# power button control function
def on_button6_clicked(b):
    with output_1:
        value = 346
        print("p_forward:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)
        
button6.on_click(on_button6_clicked)

# power control slider
def on_slider2_change(change):
    with output_1:
        print("power:",change['new'])
        servo.set_pwm(1,((int)(change['new'])))

slider2.observe(on_slider2_change, names='value')

#clear output funcition
def on_button7_clicked(b):
    output_1.clear_output()
    output_2.clear_output()
    pass

button7.on_click(on_button7_clicked)

# manual/auto
def on_button8_clicked(change):
    with output_1:
        print('auto:',change['new'])


button8.observe(on_button8_clicked, 'value')

def auto(change):
    with output_1:
        if(change['new']==True):
            print("loop:",change['new'])
            execute_thread_loop = threading.Thread(target=loop, args=(button8,))
            execute_thread_loop.start()
        
        
def loop(button8):
    cap = cv2.VideoCapture('nvarguscamerasrc ! video/x-raw(memory:NVMM), width=3280, height=2464, format=NV12,framerate=21/1 ! nvvidconv flip-method=0 ! video/x-raw, width=320, height=240, format=BGRx ! videoconvert ! video/x-raw,format=BGR ! appsink')
    # GPS init
    init_gps_GPRMC()

    # IMU init
    imu = mpu6050(0x68)
    aceleracao_max = 0
    desaceleracao_max = 0

    # ADC init
    ads = ADS1115.ADS1115()

    while(button8.value==True):
        time.sleep(0.5)
        start = time.time()
        ret, frame = cap.read()
        while not ret:
            ret, frame = cap.read()   
        _,ret_array = cv2.imencode('.jpg', frame)
        img = IPython.display.Image(data=ret_array)
        with output_2:
            output_2.clear_output()
            IPython.display.display(img)
        
        with output_3:
            output_3.clear_output()
            print("output_3")
            
            print("GPS:{}".format(read_gps()))
            print("imu:")
            aceleracao_data = imu.get_accel_data()
            aceleracao_instantanea = aceleracao_data['y']
            print("aceleracao longitudinal instantanea : {:.3f} m/s^2\n".format(aceleracao_instantanea))
            if aceleracao_data['y']>aceleracao_max:
                aceleracao_max = aceleracao_data['y']
            print("aceleracao longitudinal maxima: {:.3f} m/s^2\n".format(aceleracao_max))
            if aceleracao_data['y']<desaceleracao_max:
                desaceleracao_max = aceleracao_data['y']
            print("desaceleração longitudinal maxima: {:.3f} m/s^2\n".format(desaceleracao_max))
            volt0 = 3*(ads.readADCSingleEnded()/1000)
            volt1 = (((ads.readADCSingleEnded(1)/1000)/0.185)-7.07415)*1.8
            volt2 = (((ads.readADCSingleEnded(2)/1000)/0.185)-7.1175)*2.115
            print("Tensão da bateria: {:.3f} V\n".format(volt0))
            print("Corrente do motor: {:.3} A\n".format(volt1))
            print("Corrente do computador: {:.3f} A\n".format(volt2))
            lateral = servo.get_pwm(0)
            longitudinal = servo.get_pwm(1)
            print("Controle lateral: {}\n".format(lateral))
            print("Controle longitudinal: {}\n".format(longitudinal))
            
            end = time.time()
            print("time_loop: ",(end-start))
            
            
            
            
    cap.release()
    with output_1:
        value = 330
        print("stop:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)
        value = 307
        print("s_forward:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)


        
button8.observe(auto,'value')



Output()